In [64]:
import emoji
import pandas as pd
import numpy as np

In [65]:
emoji_dictionary={
    "0":"\u2764\uFE0F",
    "1":":baseball:",
    "2":":grinning_face_with_big_eyes:",
    "3":":disappointed_face:",
    "4":":fork_and_knife:",
}

In [66]:
for e in emoji_dictionary.values():
    print(emoji.emojize(e))

❤️
⚾
😃
😞
🍴


In [67]:
df=pd.read_csv('C:/Users/91885/Python programs/train.csv',header=None)
df1=pd.read_csv('C:/Users/91885/Python programs/test.csv',header=None)
X=pd.DataFrame.to_numpy(df)
X1=pd.DataFrame.to_numpy(df1)

In [68]:
print(X1.shape)

(56, 2)


In [69]:
x_train=X[:,1]
y_train=X[:,2]
x_test=X1[:,0]
y_test=X1[:,1]
print(y_test)

[4 3 2 0 2 0 0 0 4 2 1 3 3 2 1 3 3 2 3 4 0 2 4 3 3 3 3 0 2 0 0 1 3 2 0 1 2
 4 4 2 2 2 0 1 2 0 2 2 2 3 3 0 3 2 2 3]


In [70]:
for i in range(5):
    print(x_train[i],emoji.emojize(emoji_dictionary[str(y_train[i])]))

never talk to me again 😞
I am proud of your achievements 😃
It is the worst day in my life 😞
Miss you so much ❤️
food is life 🍴


In [71]:
word_embeddings={}
f=open('glove.6B.50d.txt',encoding='utf-8')
for line in f:
    values=line.split()
    word=values[0]
    coeffs=np.asarray(values[1:],dtype='float')
    word_embeddings[word]=coeffs
    #print(word,coeffs)
f.close()

**Step 4 Converting sentence into a vector** 

In [72]:
def embedding_output(X):
    #X.shape[0] actually gives batch size or number of sentences
    #ideally embedding_out should be (num_of_sentences*max_words_in_sentence*embedding of each word)
    max_len=10
    emb_dim=50
    embedding_out=np.zeros((X.shape[0],max_len,emb_dim))
    for ix in range(X.shape[0]):# X.shape[0] gives number of sentences
        X[ix]=X[ix].split()
        
        for ij in range(len(X[ix])):
            try:
               embedding_out[ix][ij]=word_embeddings[X[ix][ij].lower()]
            except:
               embedding_out[ix][ij]=np.zeros((50,)) 
    return embedding_out

In [73]:
embeddings_matrix_train=embedding_output(x_train)


In [74]:
embeddings_matrix_test=embedding_output(x_test)

In [75]:
#embeddings_matrix_train_y=embedding_output(y_train)
print(embeddings_matrix_test.shape)

(56, 10, 50)


** LSTM Architecture - Keras **

In [76]:
from keras.utils import to_categorical

In [77]:
y_train=to_categorical(y_train,num_classes=5)
print(y_train.shape)

(132, 5)


In [78]:
y_test=to_categorical(y_test,num_classes=5)
print(y_test.shape)

(56, 5)


In [79]:
from keras.models import Sequential
from keras.layers import *

In [88]:
model=Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64,return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 10, 64)            29440     
_________________________________________________________________
dropout_4 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_4 (Activation)    (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
_________________________________________________________________


In [89]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint


checkpoint=ModelCheckpoint("best_model.h5",monitor='val_loss',verbose=True,save_best_only=True)
earlystop=EarlyStopping(monitor='val_acc',patience=10)
hist=model.fit(embeddings_matrix_train,y_train,epochs=100,batch_size=64,shuffle=True,validation_split=0.2)

Train on 105 samples, validate on 27 samples
Epoch 1/100
105/105 [==============================] - 4s 42ms/step - loss: 1.6196 - acc: 0.2476 - val_loss: 1.5924 - val_acc: 0.2222
Epoch 2/100
105/105 [==============================] - 0s 648us/step - loss: 1.5728 - acc: 0.3333 - val_loss: 1.5965 - val_acc: 0.2222
Epoch 3/100
105/105 [==============================] - 0s 669us/step - loss: 1.5452 - acc: 0.4000 - val_loss: 1.6060 - val_acc: 0.2222
Epoch 4/100
105/105 [==============================] - 0s 743us/step - loss: 1.5199 - acc: 0.3810 - val_loss: 1.6221 - val_acc: 0.2222
Epoch 5/100
105/105 [==============================] - 0s 922us/step - loss: 1.4761 - acc: 0.4190 - val_loss: 1.6401 - val_acc: 0.2222
Epoch 6/100
105/105 [==============================] - 0s 967us/step - loss: 1.4657 - acc: 0.4286 - val_loss: 1.6622 - val_acc: 0.2222
Epoch 7/100
105/105 [==============================] - 0s 904us/step - loss: 1.4338 - acc: 0.4095 - val_loss: 1.6828 - val_acc: 0.1852
Epoch 8/100

105/105 [==============================] - 0s 684us/step - loss: 0.0837 - acc: 0.9905 - val_loss: 1.5151 - val_acc: 0.6296
Epoch 62/100
105/105 [==============================] - 0s 924us/step - loss: 0.0805 - acc: 0.9905 - val_loss: 1.4117 - val_acc: 0.6667
Epoch 63/100
105/105 [==============================] - 0s 913us/step - loss: 0.0890 - acc: 0.9714 - val_loss: 1.4567 - val_acc: 0.5926
Epoch 64/100
105/105 [==============================] - 0s 947us/step - loss: 0.0950 - acc: 0.9714 - val_loss: 1.5560 - val_acc: 0.5926
Epoch 65/100
105/105 [==============================] - 0s 759us/step - loss: 0.0558 - acc: 0.9905 - val_loss: 1.5086 - val_acc: 0.6296
Epoch 66/100
105/105 [==============================] - 0s 688us/step - loss: 0.0637 - acc: 0.9905 - val_loss: 1.5544 - val_acc: 0.6296
Epoch 67/100
105/105 [==============================] - 0s 688us/step - loss: 0.0829 - acc: 0.9810 - val_loss: 1.5017 - val_acc: 0.6667
Epoch 68/100
105/105 [==============================] - 0s 63

In [86]:
model.load_weights("best_model.h5")

OSError: Unable to open file (Unable to open file: name = 'best_model.h5', errno = 2, error message = 'no such file or directory', flags = 0, o_flags = 0)

In [90]:
model.evaluate(embeddings_matrix_test,y_test)

56/56 [==============================] - 0s 315us/step


[2.1544689621244157, 0.5892857228006635]